In [ ]:
!pip install --upgrade transformers
!pip install soynlp
!pip install sentencepiece

     |████████████████████████████████| 1.5MB 21.8MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
     |████████████████████████████████| 2.9MB 41.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3481f161a2da5b29881fd884097dd693bbf91c4e68a1231fab081839775a1ed3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 419kB 21.6MB/s 
     |████████████████████████████████| 1.1MB 22.1MB/s 


In [ ]:
import argparse
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
from tqdm import tqdm
import os
import random
import time
import datetime

#추가
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
''' ipynb can't use argparse!
parser = argparse.ArgumentParser(description = 'prediction')

parser.add_argument('--input_text', type=str, default='test.txt')
parser.add_argument('--output_text', type=str, default='result.txt')

## checkpoint path
parser.add_argument('--checkpoint_path_new', type=str, default='best_new_kcbert_wd0.001.pth')
parser.add_argument('--checkpoint_path_mid', type=str, default='best_mid_kcbert_wd0.001.pth')
parser.add_argument('--checkpoint_path_lbl', type=str, default='kcbert_best_lbl.pth')

## index data
parser.add_argument('--mid_index_path', type=str, default='mid_index.txt')
parser.add_argument('--new_index_path', type=str, default='new_index.txt')
parser.add_argument('--label_convert_path', type=str, default='label_intnchar.txt')

args = parser.parse_args()
'''

import easydict

args = easydict.EasyDict({
    "input_text": "test.txt",
    "output_text": "result.txt",
    
    "checkpoint_path_new": "best_new_kcbert_wd0.001.pth",
    "checkpoint_path_mid": "best_mid_kcbert_wd0.001.pth",
    "checkpoint_path_lbl": "kcbert_best_lbl.pth",

    "mid_index_path": "mid_index.txt",
    "new_index_path": "new_index.txt",
    "label_convert_path": "label_intnchar.txt"
})

In [ ]:
## Load data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# train = pd.read_table("/content/drive/MyDrive/NLP_MZ/"+"train_preprocess_lbl_ver2.txt")
test = pd.read_table("/content/drive/MyDrive/NLP_MZ/"+args.input_text, header = None)
test.columns = ['document']

In [ ]:
#### Cleaning text (데이터 전처리) ####

# removing special characters
test['document'] = test['document'].str.replace(r'[-=+,#/\?:^$.@*\"※~>`\'…》]', " ", regex=True)
test['document'] = test['document'].str.replace(r'\t+', " ", regex=True)
test['document'] = test['document'].str.replace(r'[\\n]+'," ", regex=True)
test['document'] = test['document'].str.replace(r'[-+]?\d+'," ", regex=True)

# normalize repeat words (반복어 제거)
from soynlp.normalizer import *

for i in range(len(test['document'])):
  test['document'][i] = repeat_normalize(test['document'][i], num_repeats=1)

# removing stop_words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')

stop_words = "아 휴 아이구 아이고 아이쿠 아이코 깍 히히 꺽 낑 깡 히 하 하하 어 나 우리 저희 따라 의해 을 를 뿐이다 의거하여 근거하여 입각하여 예하면 예를 들면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 그러나 그런데 하지만 듣간에 논하지 않다 따지지 설사 비록 더라도 아니면 만 못하다 편이 불문하고 향하여 향해서 쪽으로 틈타 이용하여 타다 제외하고 외에 이 이밖에 하여야 비로소 한다면 몰라도 외에도 이곳 이리하여 그리하여 그렇게 함으로써 일때 할때 앞에서 중에서 보는데 으로써 로써 까지 일것이다 반드시 임에 할수있다 할수있어 틀림없다 등 등등 등등등 등등등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 여 좀 조금 다수 하물며 또한 그러나 그렇지만 뿐이다 까악 만약 툭 딱 삐걱 보드득 비걱 꽈당 응당 각각 제각 와 과 고로 까닭에 아니나다를까 이지만 거니와 진짜로 하하하 하하 허허 허허허 아하 거봐 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 더군다나 하물며 더욱이 야 이봐 어이 여보시오 흐흐 흐흐흐 흥 흥흥 휴 휴휴 헉 헉헉 헐떡헐떡 영차 영차영 여차 어기여차 끙끙 아야 아 앗 콸콸 괄괄 졸졸 줄줄 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 바꾸어말하면 바꾸어말하자면 또 혹시 혹은 답다 및 그에 설령 가령 이젠 어찌됐든 어찌됏든 비추어보아 비교적 좀 당장 하자마자 바와같이 허 헉 허걱 와르르 팍 팍팍 퍽 퍽퍽 펄렁 양자 마저 가까스로 즈음하여 다른 개의치않고 무릎쓰고 말할것도없고 심지어 조차도 몰랐다 모른다 겸사겸사 그 너희 너희들 그들 붕붕 윙윙 공동으로 너 타인 것 것들 우리 엉엉 휘익 오호 아하 흐흐 쉿 상대적으로 절대적으로 한항목 여부 운운 다시말하면 이러이러하다 하도 다음으로 오히려 어찌됐어 어때 어째서 요만큼 요만한 이렇구나 끼익 삐걱 따위 부류 혼자 종합한것과같이 총적으로 으로서 참 그만이다 쿵 탕탕 쿵쿵 탕 쾅쾅 쾅 둥 둥둥 봐 아이야 아니 와 와아 응 아이 참나 참 되 말하 알 그녀 그냥 줘 가장 차선 차선으로 최고 짱 돼 해 "
stop_words = stop_words.split(' ')

for i in range(len(test['document'])):
  result = []
  word_tokens = word_tokenize(test['document'][i])
  for w in word_tokens:
    if w not in stop_words:
      result.append(w)
  word = ' '.join(result)
  test['document'][i] = word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# train.head()
test.head()

,document
0,내 앞에 있는 차 계속 따라가
1,차 따라붙기 가능하냐
2,앞차와 제동거리를 유지하면서 따라가
3,앞차 안 붙어
4,앞에 차 뒤에 붙어줄래


In [ ]:
import logging
import unicodedata
from shutil import copyfile
import sentencepiece as spm
from transformers import PreTrainedTokenizer
 
 
logger = logging.getLogger(__name__)
 
VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}
 
PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}
 
PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}
 
PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}
 
SPIECE_UNDERLINE = u'▁'
 
 
class KoBertTokenizer(PreTrainedTokenizer):

    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES
 
    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )
 
        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)
 
       # self.max_len_single_sentence = self.model_max_length - 2  # take into account special tokens
       # self.max_len_sentences_pair = self.model_max_length - 3  # take into account special tokens
 
        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt
 
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)
 
    @property
    def vocab_size(self):
        return len(self.idx2token)
 
    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state
 
    def __setstate__(self, d):
        self.__dict__ = d
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)
 
    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = "".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')
 
        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()
 
        return outputs
 
    def _tokenize(self, text, return_unicode=True, sample=False):

        text = self.preprocess_text(text)
 
        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)
 
        return new_pieces
 
    def _convert_token_to_id(self, token):
       
        return self.token2idx.get(token, self.token2idx[self.unk_token])
 
    def _convert_id_to_token(self, index, return_unicode=True):
        
        return self.idx2token[index]
 
    def convert_tokens_to_string(self, tokens):
       
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string
 
    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep
 
    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
 
        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))
 
        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]
 
    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
      
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]
 
    def save_vocabulary(self, save_directory):

        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return
 
        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])
 
        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)
 
        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1
 
        return out_vocab_model, out_vocab_txt


In [ ]:
#tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') 

tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-large')

In [ ]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 32 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode("[CLS] " + data_df[DATA_COLUMN][i] + " [SEP]", max_length=SEQ_LEN, pad_to_max_length=True)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN
 
        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답을 targets 변수에 저장해 줌
        # targets.append(data_df[LABEL_COLUMN][i])
 
    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    print(tokens.shape)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)
 
    return [tokens, masks, segments], targets
 
# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    # data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y
 
SEQ_LEN = 32

# 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 의도를 포함하고 있는 칼럼
# LABEL_COLUMN = "label"


In [ ]:
# test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

  0%|          | 0/9228 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 9228/9228 [00:01<00:00, 4848.32it/s]


(9228, 32)


In [ ]:

test_inputs = torch.Tensor(test_x[0])
# test_labels = torch.Tensor(test_y).type(torch.LongTensor)
test_masks = torch.Tensor(test_x[1])

print(test_inputs[0])
# print(test_labels[0])
print(test_masks[0])


tensor([2.0000e+00, 2.0000e+00, 6.0900e+02, 1.1310e+04, 8.0320e+03, 2.8390e+03,
        8.2490e+03, 2.7858e+04, 3.0000e+00, 3.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
# 배치 사이즈
batch_size = 1

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 배치 사이즈 만큼 데이터를 가져옴

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# 분류를 위한 BERT 모델 생성
# model = BertForSequenceClassification.from_pretrained('beomi/kcbert-large', num_labels=784)

model_new = BertForSequenceClassification.from_pretrained('beomi/kcbert-large', num_labels=37)
model_mid = BertForSequenceClassification.from_pretrained('beomi/kcbert-large', num_labels=160)
model_lbl = BertForSequenceClassification.from_pretrained('beomi/kcbert-large', num_labels=784)


Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [ ]:
model_new.zero_grad()
model_new.to(device)

model_mid.zero_grad()
model_mid.to(device)

model_new.zero_grad()
model_new.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 1024, padding_idx=0)
      (position_embeddings): Embedding(300, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
### prediction.py

checkpoint_path_new = '/content/drive/MyDrive/NLP_MZ/checkpoints_yb/' + args.checkpoint_path_new
checkpoint_path_mid = '/content/drive/MyDrive/NLP_MZ/checkpoint_yw/' + args.checkpoint_path_mid
checkpoint_path_lbl = '/content/drive/MyDrive/NLP_MZ/checkpoints_jy/' + args.checkpoint_path_lbl

mid_index_path = "/content/drive/MyDrive/NLP_MZ/" + args.mid_index_path
new_index_path = "/content/drive/MyDrive/NLP_MZ/" + args.new_index_path

list_of_lists_mid = []
list_of_lists_new = []

with open(mid_index_path, encoding = "utf-8") as f:
  for line in f:
    inner_list_mid = [elt.strip() for elt in line.split(',')]
    list_of_lists_mid.append(inner_list_mid)

mid_index = list_of_lists_mid

with open(new_index_path, encoding = "utf-8") as f:
  for line_new in f:
    inner_list_new = [elt.strip() for elt in line_new.split(',')]
    list_of_lists_new.append(inner_list_new)

new_index = list_of_lists_new

# fine_tuned 모델 불러오기
if torch.cuda.is_available():
    model_new.load_state_dict(torch.load(checkpoint_path_new))
    model_mid.load_state_dict(torch.load(checkpoint_path_mid))
    model_lbl.load_state_dict(torch.load(checkpoint_path_lbl))
else:
    model_new.load_state_dict(torch.load(checkpoint_path_new, map_location=device))
    model_mid.load_state_dict(torch.load(checkpoint_path_mid, map_location=device))
    model_lbl.load_state_dict(torch.load(checkpoint_path_lbl, map_location=device))

model_new.to(device)
model_mid.to(device)
model_lbl.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 1024, padding_idx=0)
      (position_embeddings): Embedding(300, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [ ]:
### test ###

print("")
print("Running test...")

  #시작 시간 설정
t0 = time.time()

  # 평가모드로 변경
model_new.eval()
model_mid.eval()
model_lbl.eval()

  # 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 각 분류기의 weight
weight_new = 1.0
weight_mid = 2.0
weight_lbl = 3.0

preds = None

  # 데이터로더에서 배치만큼 반복하여 가져옴
for batch in test_dataloader:

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask = batch
    b_input_ids = b_input_ids.type(torch.LongTensor).to(device)
    b_input_mask =  b_input_mask.type(torch.LongTensor).to(device)
    # b_labels = b_labels.type(torch.LongTensor).to(device)
      
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행 (각 분류기의 아웃풋 구하기)
        outputs_new = model_new(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask)
        outputs_mid = model_mid(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask)
        outputs_lbl = model_lbl(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask)
    

    ##### resize_output! #####
    # print(outputs_mid)

    logits_mid = outputs_mid[0]
    logits_mid = logits_mid.detach().cpu().numpy()

    # print(logits_mid)
    large_outputs_mid = np.zeros([1, 784])
    for idx, acc in enumerate(logits_mid[0]):
      for idx2, place in enumerate(mid_index):
        if idx==idx2:
          for ind in place:
            ind = int(ind)
            large_outputs_mid[0][ind] = acc

    logits_new = outputs_new[0]
    logits_new = logits_new.detach().cpu().numpy()

    large_outputs_new = np.zeros([1, 784])
    for idx_new, acc_new in enumerate(logits_new[0]):
      for idx2_new, place_new in enumerate(new_index):
        if idx_new==idx2_new:
          for ind_new in place_new:
            ind_new = int(ind_new)
            large_outputs_new[0][ind_new] = acc_new

    logits_lbl = outputs_lbl[0]
    logits_lbl = logits_lbl.detach().cpu().numpy()

    # 각 분류기의 아웃풋에 weight를 곱해 최종 예측값 구하기
    outputs_final = weight_new * large_outputs_new + weight_mid * large_outputs_mid + weight_lbl * logits_lbl
    # outputs_final = large_outputs_new * large_outputs_mid * logits_lbl

    logits = outputs_final

    # CPU로 데이터 이동
    # label_ids = b_labels.to('cpu').numpy()

    if preds is None:
      preds = logits
    else:
      preds = np.append(preds, logits, axis = 0)

    # tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    # eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

    print('test num: {}...'.format(nb_eval_steps))

# cur_acc = eval_accuracy/nb_eval_steps

preds = np.argmax(preds, axis = 1)
# with open(output_file, "w", encoding="utf-8") as f:
#   for pred in preds:
#     f.write("{}\n".format(pred))

# print("  Accuracy: {0:.8f}".format(eval_accuracy/nb_eval_steps))
print("  Test took: {:}".format(format_time(time.time() - t0)))

print("")
print("Test complete!")

Streaming output truncated to the last 5000 lines.
test num: 4232...
test num: 4233...
test num: 4234...
test num: 4235...
test num: 4236...
test num: 4237...
test num: 4238...
test num: 4239...
test num: 4240...
test num: 4241...
test num: 4242...
test num: 4243...
test num: 4244...
test num: 4245...
test num: 4246...
test num: 4247...
test num: 4248...
test num: 4249...
test num: 4250...
test num: 4251...
test num: 4252...
test num: 4253...
test num: 4254...
test num: 4255...
test num: 4256...
test num: 4257...
test num: 4258...
test num: 4259...
test num: 4260...
test num: 4261...
test num: 4262...
test num: 4263...
test num: 4264...
test num: 4265...
test num: 4266...
test num: 4267...
test num: 4268...
test num: 4269...
test num: 4270...
test num: 4271...
test num: 4272...
test num: 4273...
test num: 4274...
test num: 4275...
test num: 4276...
test num: 4277...
test num: 4278...
test num: 4279...
test num: 4280...
test num: 4281...
test num: 4282...
test num: 4283...
test num: 428

In [ ]:
### 나온 예측값(정수형 label) 문자열 형태로 변환하기

# 정수형 label 불러오기
col_name = ['label']
preds = pd.DataFrame(preds, columns = col_name)

# 정수형-문자열 label 매칭한 파일
label_list = pd.read_table('/content/drive/MyDrive/NLP_MZ/' + args.label_convert_path)

# dev_int = dev_int.loc[:, ['label', 'document']]
label_list = label_list.loc[:, ['label_int', 'label_char']]
dict_label = label_list.set_index('label_int').T.to_dict('list')

# 정수형-문자열 label 매칭 파일에 맞게 변환하기
preds_txt = preds.replace({'label': dict_label})
preds_txt_file = preds_txt
# result.txt 저장
preds_txt_file.to_csv('/content/drive/MyDrive/NLP_MZ/' + args.output_text, index=False)


In [ ]:
preds_txt

,label
0,차량제어-운전제어-일자따라가기
1,차량제어-운전제어-일자따라가기
2,차량제어-운전제어-일자따라가기
3,차량제어-운전제어-일자따라가기
4,차량제어-운전제어-일자따라가기
...,...
9223,교육-놀이&운동서비스-주사위게임실행
9224,안심케어-메시지-가족에게자동전화
9225,엔터테인먼트-감성대화-부정적변화반응
9226,안심케어-메시지-긴급메시지취소
